In [1]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [2]:
import galois
import numpy as np
import time

In [47]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(127)

In [48]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [40]:
#ho verificato che funziona pure con i random
#A = field.Random((k, n_k))
#print(A)

#questa è con l'esempio di 4.3
A = field([[0,0,1,1],
           [0,0,1,2],
           [1,1,0,2]])

In [41]:
matrice_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrice_bin1 = field(matrice_bin1)

matrice_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrice_bin2 = field(matrice_bin2)

In [42]:
A_primo = field(matrice_bin1 @ A @ matrice_bin2)
print(A_primo)

[[1 2 1 0]
 [0 1 0 1]
 [0 2 0 1]]


In [8]:
A_primo = field([[2,2,0,3],
           [1,4,7,2],
           [0,3,1,2]])

In [45]:
#definito funzione ordinamento lessicografico riga (ordino ogni riga e definisco ordine degli indici) e gestiti casi di 
#conflitto. L'argomento colonne è quello che ci indica in che ordine verificare i conflitti 
#ex: colonne=[0,1,2] --> ordino per colonna 0, se c'è conflitto tra due vedo la colonna 1 e cosi via

#faccio ordinamento crescente dei valori all'interno della riga e torno matrice con righe ordinate (ordinato).
#da questa matrice faccio l'ordinamento elemento a elemento posizionale corrispondente e definisco quindi gli indici 
#riordinati. ex: [1 0 2] --> prima la riga ORIGINALE 1 poi riga ORIGINALE 0 e così via. Per originale intendo "matrice"
def ordina_matrice(matrice):
    #righe matrice
    n = matrice.shape[0]  
    #contatore confronti
    confronti = 0  
    #array indici originali
    indici = np.arange(n)  

    #bubble sort per conteggio confronti
    for i in range(n):
        #confronto riga i con tutte le j
        for j in range(i + 1, n):
            #scorro tra tutte le colonne
            for k in range(matrice.shape[1]):  
                #conta confronto (almeno uno lo fa)
                confronti += 1
                #se riga i minore, esci dal ciclo
                if matrice[i, k] < matrice[j, k]:
                    break  
                #se non è minore vedo se è maggiore e scambio righe ed indici
                elif matrice[i, k] > matrice[j, k]:
                    matrice[[i, j]] = matrice[[j, i]] #AI FINI DELLA VELOCITÀ LO POSSO TOGLIERE
                    indici[[i, j]] = indici[[j, i]]
                    break 
                #se entro in questo if vuol dire che praticamente sono uguali fino all'ultima colonna
                if k == (n_k-1):
                    print("Protocollo non funzionerà")
                #se uguali continua a confrontare la prossima colonna

    print("Matrice ordinata:\n", matrice) #QUESTO AI FINI DI VELOCITA LO POSSO TOGLIERE
    print("Indici delle righe ordinate:", indici)
    print("Numero totale di confronti:", confronti)
    
    return matrice, indici, confronti



def ordina_per_riga(matrice, colonne):
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
        
    ordinato = np.array(ordinato)
    
    mat_ok, indici_ordinamento, num_confronti = ordina_matrice(ordinato)
    print("\n",indici_ordinamento)
    print("Matrice ordinata per righe: ",matrice[indici_ordinamento])
    return matrice[indici_ordinamento]

#ordino poi per il primo elemento della colonna e in caso di conflitti li ho gestiti. 
def ordina_per_colonna(matrice, righe):
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
    return matrice[:, indici_ordinamento]

#qui faccio il test
def test_uguaglianza(matrice1, matrice2):
    colonne = range(n_k)
    righe = range(k)
    print("Matrice1: ", matrice1)

    ordinata1 = ordina_per_colonna(ordina_per_riga(matrice1, colonne), righe)
    print("\nOrdinata 1: ",ordinata1)

    print("Matrice2: ", matrice2)
    ordinata2 = ordina_per_colonna(ordina_per_riga(matrice2, colonne), righe)
    print("\nOrdinata2: ",ordinata2)     

    if np.array_equal(ordinata1, ordinata2):
        print("Le due matrici hanno la stessa CF.")
        return True
    else:
        print("Le due matrici hanno CF diversa")
        return False   



In [46]:
test_uguaglianza(A, A_primo)

Matrice1:  [[0 0 1 1]
 [0 0 1 2]
 [1 1 0 2]]
Matrice ordinata:
 [[0 0 1 1]
 [0 0 1 2]
 [0 1 1 2]]
Indici delle righe ordinate: [0 1 2]
Numero totale di confronti: 8

 [0 1 2]
Matrice ordinata per righe:  [[0 0 1 1]
 [0 0 1 2]
 [1 1 0 2]]

Ordinata 1:  [[0 0 1 1]
 [0 0 1 2]
 [1 1 0 2]]
Matrice2:  [[1 2 1 0]
 [0 1 0 1]
 [0 2 0 1]]
Matrice ordinata:
 [[0 0 1 1]
 [0 0 1 2]
 [0 1 1 2]]
Indici delle righe ordinate: [1 2 0]
Numero totale di confronti: 8

 [1 2 0]
Matrice ordinata per righe:  [[0 1 0 1]
 [0 2 0 1]
 [1 2 1 0]]

Ordinata2:  [[0 0 1 1]
 [0 0 1 2]
 [1 1 0 2]]
Le due matrici hanno la stessa CF.


True

In [49]:
#FACCIO METODO DI MONTECARLO
#numero iterazioni Monte Carlo
num_iterazioni = 10000  

#QUESTA POTREBBE NON SERVIRE
# Funzione per generare matrici binarie casuali
def genera_matrice_binaria(dimensioni, campo):
    matrice_binaria = np.zeros(dimensioni, dtype = int)
    perm = np.random.permutation(dimensioni[0])
    for i in range(dimensioni[0]):
        matrice_binaria[i, perm[i]] = 1
    return campo(matrice_binaria)

#fa il procedimento di ordinamento tra le righe (che pero sono già ordinate al loro interno)
def ordina_matrice(matrice):
    #righe matrice
    n = matrice.shape[0]  
    #contatore confronti
    confronti = 0  
    #array indici originali
    indici = np.arange(n)  

    #bubble sort per conteggio confronti
    for i in range(n):
        #confronto riga i con tutte le j
        for j in range(i + 1, n):
            #scorro tra tutte le colonne
            for k in range(matrice.shape[1]):  
                #conta confronto (almeno uno lo fa)
                confronti += 1
                #se riga i minore, esci dal ciclo
                if matrice[i, k] < matrice[j, k]:
                    break  
                #se non è minore vedo se è maggiore e scambio righe ed indici
                elif matrice[i, k] > matrice[j, k]:
                    matrice[[i, j]] = matrice[[j, i]] #AI FINI DELLA VELOCITÀ LO POSSO TOGLIERE
                    indici[[i, j]] = indici[[j, i]]
                    break 
                #se entro in questo if vuol dire che praticamente sono uguali fino all'ultima colonna
                if k == (n_k-1):
                    print("Protocollo non funzionerà")
                #se uguali continua a confrontare la prossima colonna

    #print("Matrice ordinata:\n", matrice) #QUESTO AI FINI DI VELOCITA LO POSSO TOGLIERE
    #print("Indici delle righe ordinate:", indici)
    #print("Numero totale di confronti:", confronti)
    #POSSO  EVITARE IL RETURN DELLA MATRICE
    return matrice, indici, confronti
# Funzione per ordinare le righe con contatori di ordinamenti e swap (TOGLI COLONNE)
def ordina_per_riga_con_contatori(matrice, colonne, num_swap_righe, num_confronti):
    # Mantieni traccia della posizione originale delle righe
    posizioni_originali = np.arange(matrice.shape[0])
    
    # Ordina gli elementi all'interno di ogni riga
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
    
    ordinato = np.array(ordinato)
    mat_ok, indici_ordinamento, num_confronti = ordina_matrice(ordinato)
    # Trova gli indici di ordinamento usando np.lexsort sulle colonne
    #indici_ordinamento = np.lexsort([ordinato[:, col] for col in colonne[::-1]])
    
    # Conta gli swap solo quando la posizione delle righe cambia
    for i, indice in enumerate(indici_ordinamento):
        if indice != posizioni_originali[i]:
            num_swap_righe += 1
    
    # Riordina la matrice in base agli indici trovati
    matrice_ordinata = matrice[indici_ordinamento]
    
    return matrice_ordinata, num_swap_righe, num_confronti


#POTREI DOVER IMPLEMENTARE UNA COSA SIMILE A SOPRA PER IL COUNT DEI CONFRONTI (LEXSORT NON CE L'HA)
#ordinare le colonne con contatori di ordinamenti e swap
def ordina_per_colonna_con_contatori(matrice, righe, num_swap_colonne):
    # Mantieni traccia della posizione originale delle colonne
    posizioni_originali = np.arange(matrice.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, indice in enumerate(indici_ordinamento):
        if indice != posizioni_originali[i]:
            num_swap_colonne += 1
    
    # Riordina la matrice in base agli indici trovati
    matrice_ordinata = matrice[:, indici_ordinamento]
    # Faccio num_swap_colonne/2 poichè contando il numero di diversi ne conta ovviamente il doppio. 
    # Ex: (0, 1, 2, 3) potrebbe diventare (0, 1, 3, 2) il contatore sarebbe a due, ma io ho fatto uno swap
    return matrice_ordinata, num_swap_colonne

#IL NOME DELLA FUNZIONE È FORVIANTE E POSSO ELIMINARE MATRICE 2 PERCHÈ NON C'È NESSUN CONFRONTO
def test_uguaglianza_con_contatori(matrice1, matrice2, num_swap_righe, num_swap_colonne, num_confronti):
    start_time = time.perf_counter()
    #intervallo per le colonne
    colonne = range(n_k)  
    #intervallo corretto per le righe
    righe = range(k)      
    # Ordinamento matrice1
    ordinata1, num_swap_righe, num_confronti = ordina_per_riga_con_contatori(matrice1, colonne, num_swap_righe, num_confronti)
    cf1, num_swap_colonne = ordina_per_colonna_con_contatori(ordinata1, righe, num_swap_colonne)
    
    # Ordinamento matrice2
    #ordinata2, num_swap_righe = ordina_per_riga_con_contatori(matrice2, colonne, num_swap_righe, num_confronti)
    #cf2, num_swap_colonne = ordina_per_colonna_con_contatori(ordinata2, righe, num_swap_colonne)
    end_time = time.perf_counter()
    durata = (end_time - start_time)*1000
    # Confronto
    # Vedo prima se prese le due matrici a meno di due righe sono uguali E poi se effettivamente sono uguali allora ritorno True
    #sono un coglione tanto l'uguaglianza è dimostrata, DEVO EVITARE DI CONSIDERARE LE DUE RIGHE SULL'RODINAMENTO DELLE RIGHE
    #TUTTA L'ANALISI DI MONTECARLO NON LA DEVO FARE QUA, MA LA DEVO FARE SULLA FUNZIONE DEL CONFRONTA ELEMENTO A ELEMENTO PER 
    #EVITARE DI VEDERLI TUTTI
    # QUINDI UNA VOLTA CHE HO LA MATRICE A DEVO FARE IL PROCESSO DI ORDINAMENTO DELLE RIGHE O SENZA CONSIDERARE GLI ULTIMI DUE 
    # ELEMENTI DELLA COLONNA (PER ESEMPIO) OPPURE CONSIDERANDOLI E POI NON ANDANDOLI A CONFRONTARE AD UNO A UNO CON QUELLI 
    # DELLA RIGA IN CONFLITTO
    return cf1, num_swap_righe, num_swap_colonne, num_confronti, durata

# Funzione per eseguire la simulazione di Monte Carlo
def simulazione_montecarlo(num_iterazioni):
    #contatore simulazioni positive
    simulazioni_positive = 0
    #simulazione accettabile
    simulazione_accettabile = 0
    #contatore swap righe
    num_swap_righe = 0  
    #contatore swap colonne
    num_swap_colonne = 0  
    #contatore confronti righe
    num_confronti = 0
    #inizializzo la durata
    tempo_esecuzione = 0.0

    
    for _ in range(num_iterazioni):
        #field.order mi fa generare i numeri da 0 a 7 (field.order = 8)
        A = field(np.random.randint(0, field.order, size=(k, n_k)))
        #matrice_bin1 = genera_matrice_binaria((k, k), field)
        #matrice_bin2 = genera_matrice_binaria((n_k, n_k), field)

        #A_primo = field(matrice_bin1 @ A @ matrice_bin2)

        #faccio il test di uguaglianza tra A e A_primo
        uguaglianza, num_swap_righe, num_swap_colonne, num_confronti, durata = test_uguaglianza_con_contatori(A, A_primo, num_swap_righe, num_swap_colonne, num_confronti)
        tempo_esecuzione += durata
        
        #if uguaglianza:
            #simulazioni_positive += 1
        #    simulazione_accettabile += 1
                
        
    # Output statistiche
    print()
    print(f"Tempo esecuzione medio: {tempo_esecuzione/num_iterazioni} ms")
    print(f"Numero medio di swap righe: {num_swap_righe/num_iterazioni}")
    print(f"Numero medio di swap colonne: {num_swap_colonne/num_iterazioni}")
    print(f"Numero medio di confronti tra le righe in una matrice: {num_confronti/num_iterazioni}")
    print(f"Percentuale di accettati (non controllo tutte le righe, mi fermo a x colonne prima): {(simulazione_accettabile/num_iterazioni)*100}%")

# Esegui la simulazione
simulazione_montecarlo(num_iterazioni)



Tempo esecuzione medio: 67.79098301611242 ms
Numero medio di swap righe: 125.0203
Numero medio di swap colonne: 124.9946
Numero medio di confronti tra le righe in una matrice: 2.1347
Percentuale di accettati (non controllo tutte le righe, mi fermo a x colonne prima): 0.0%
